# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as model_selection
from numpy.random import shuffle

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from sklearn.svm import SVC 
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go

%matplotlib inline

skip = True

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Archie_Short_CW_Folder_UG'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['.DS_Store', 'Personal_Dataset', 'Models', 'CW_Dataset', 'Code', 'test_functions.ipynb']


# Import and Format training data

In [ ]:
TRAIN_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/train')
tData = pd.read_csv(os.path.join(TRAIN_FOLDER,"train_data_refined.csv"))
trainData = tData.to_numpy()


In [ ]:
#length of training data
print(trainData.shape) 

print(trainData[0])
#reformat input to data array, label array 
def formatTraining(trainingData):
  labels = []
  formattedTraining = []
  for sample in trainingData:
    labels.append(int(sample[len(sample)-1]))
    sample = sample[:-1]; sample 
    formattedTraining.append(sample) 


  return formattedTraining, labels
  
shuffle(trainData)


x,y = formatTraining(trainData)

(12271, 8501)
[ 0.11125135  0.16247456  0.07009258 ... 31.         15.
  7.        ]


# RSVM Model creation and training
 

https://towardsdatascience.com/svm-classifier-and-rbf-kernel-how-to-make-better-models-in-python-73bb4914af5b

https://stackoverflow.com/questions/34668807/much-worse-performance-with-rbf-kernel-than-linear-in-svm-in-python-scikit-learn

https://medium.com/analytics-vidhya/hyperparameter-tuning-an-svm-a-demonstration-using-hyperparameter-tuning-cross-validation-on-96b05db54e5b

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)


X_train, X_test, y_train, y_test = model_selection.train_test_split(x,y, train_size=0.8, test_size=0.2)


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

skip = True
if  not skip:

#https://medium.com/analytics-vidhya/hyperparameter-tuning-an-svm-a-demonstration-using-hyperparameter-tuning-cross-validation-on-96b05db54e5b Reference 
#https://towardsdatascience.com/svm-classifier-and-rbf-kernel-how-to-make-better-models-in-python-73bb4914af5b reference 
  model = SVC(kernel='rbf', gamma=0.01,C=1000)
  model.fit(X_train, y_train) 
  y_pred = model.predict(X_test)
  accuracy_score(y_test, y_pred)
  #reference https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

  print(classification_report(y_pred,y_test))


**Hyperparmeter Tuning**
Here I followed this article on how to use gridsearch to find the optimal parameters for SVC  https://aneesha.medium.com/svm-parameter-tuning-in-scikit-learn-using-gridsearchcv-2413c02125a0 

In [ ]:
#https://aneesha.medium.com/svm-parameter-tuning-in-scikit-learn-using-gridsearchcv-2413c02125a0 reference 
def svc_param_selection(X, y, nfolds):
    Cs =  [10,100,1000]
    gammas = [0.001,0.001]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

if not skip:
  print(svc_param_selection(X_train[:1000],y_train[:1000],10))

In [ ]:
MODEL_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'Models')
import joblib

model_good = False
#SAVE
if model_good:
  joblib.dump(model, os.path.join(MODEL_FOLDER,"RSVM.joblib"))

In [ ]:
#LOAD

loaded_svm = joblib.load(os.path.join(MODEL_FOLDER,"RSVM.joblib"))
y_pred = loaded_svm.predict(X_test)
accuracy_score(y_pred,y_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       0.71      0.77      0.74       226
           2       0.06      1.00      0.11         3
           3       0.08      1.00      0.15        10
           4       0.91      0.82      0.86      1085
           5       0.56      0.65      0.60       331
           6       0.47      0.63      0.53       107
           7       0.80      0.60      0.68       693

    accuracy                           0.72      2455
   macro avg       0.51      0.78      0.53      2455
weighted avg       0.79      0.72      0.75      2455

